# LinkedIn Post Analysis

This notebook demonstrates how to use the LinkedIn scraper and analyze the extracted data.

## Setup

In [ ]:
import sys
import json
from pathlib import Path
import pandas as pd

# Add scripts to path
sys.path.append(str(Path.cwd().parent / 'scripts'))

from linkedin_scraper import LinkedInPostScraper

## 1. Scrape a LinkedIn Post

In [ ]:
# Target post URL
post_url = "https://www.linkedin.com/posts/klarna_klarnas-climate-resilience-program-activity-7346877091532959746-748v/"

# Create scraper (headless=False to see the browser)
scraper = LinkedInPostScraper(headless=False)

# Scrape the post
post_data = scraper.scrape_post(post_url, output_path="../data/json/post_data.json")

print("Scraping complete!")

## 2. Load and Explore the Data

In [ ]:
# Load the scraped data
with open('../data/json/post_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Display the data
print("Post URL:", data['url'])
print("\nAuthor:", data.get('author', 'N/A'))
print("\nPost Text:")
print(data.get('post_text', 'N/A')[:300], "...")
print("\nEngagement Metrics:")
print(f"  Likes: {data.get('likes', 'N/A')}")
print(f"  Comments: {data.get('comments', 'N/A')}")
print(f"  Shares: {data.get('shares', 'N/A')}")

## 3. Basic Sentiment Analysis

In [ ]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

if data.get('post_text'):
    # TextBlob sentiment
    blob = TextBlob(data['post_text'])
    textblob_sentiment = blob.sentiment.polarity
    
    # VADER sentiment
    vader = SentimentIntensityAnalyzer()
    vader_scores = vader.polarity_scores(data['post_text'])
    
    print("Sentiment Analysis Results:")
    print("="*50)
    print(f"TextBlob Polarity: {textblob_sentiment:.3f}")
    print(f"  (Range: -1 = negative, 0 = neutral, 1 = positive)")
    print(f"\nVADER Compound Score: {vader_scores['compound']:.3f}")
    print(f"  Positive: {vader_scores['pos']:.3f}")
    print(f"  Neutral: {vader_scores['neu']:.3f}")
    print(f"  Negative: {vader_scores['neg']:.3f}")
    
    # Interpretation
    if vader_scores['compound'] >= 0.05:
        sentiment_label = "Positive 😊"
    elif vader_scores['compound'] <= -0.05:
        sentiment_label = "Negative 😞"
    else:
        sentiment_label = "Neutral 😐"
    
    print(f"\nOverall Sentiment: {sentiment_label}")

## 4. Engagement Rate Calculation

In [ ]:
# Calculate total engagement
likes = data.get('likes', 0) or 0
comments = data.get('comments', 0) or 0
shares = data.get('shares', 0) or 0

total_engagement = likes + comments + shares

print("Engagement Breakdown:")
print("="*50)
print(f"Likes:    {likes:>6} ({likes/total_engagement*100:.1f}%)")
print(f"Comments: {comments:>6} ({comments/total_engagement*100:.1f}%)")
print(f"Shares:   {shares:>6} ({shares/total_engagement*100:.1f}%)")
print(f"\nTotal Engagement: {total_engagement}")

## 5. Visualize Engagement

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Create data for visualization
metrics = ['Likes', 'Comments', 'Shares']
values = [likes, comments, shares]
colors = ['#0077B5', '#00A0DC', '#008CC9']

# Create bar chart
bars = plt.bar(metrics, values, color=colors, alpha=0.8, edgecolor='black')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.title('LinkedIn Post Engagement Metrics', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Metric Type', fontsize=12)
plt.tight_layout()
plt.show()

## 6. Word Cloud of Post Content

In [ ]:
# Note: wordcloud is not in requirements.txt, install if needed:
# pip install wordcloud

try:
    from wordcloud import WordCloud
    
    if data.get('post_text'):
        # Create word cloud
        wordcloud = WordCloud(width=800, height=400, 
                             background_color='white',
                             colormap='viridis',
                             max_words=50).generate(data['post_text'])
        
        # Display
        plt.figure(figsize=(12, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('Post Content Word Cloud', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
except ImportError:
    print("WordCloud not installed. Run: pip install wordcloud")

## 7. Save Enhanced Analysis

In [ ]:
# Add analysis results to data
if data.get('post_text'):
    data['analysis'] = {
        'sentiment': {
            'textblob_polarity': textblob_sentiment,
            'vader_compound': vader_scores['compound'],
            'vader_positive': vader_scores['pos'],
            'vader_neutral': vader_scores['neu'],
            'vader_negative': vader_scores['neg'],
            'label': sentiment_label
        },
        'engagement': {
            'total': total_engagement,
            'likes_percentage': likes/total_engagement*100 if total_engagement > 0 else 0,
            'comments_percentage': comments/total_engagement*100 if total_engagement > 0 else 0,
            'shares_percentage': shares/total_engagement*100 if total_engagement > 0 else 0
        },
        'text_stats': {
            'character_count': len(data['post_text']),
            'word_count': len(data['post_text'].split())
        }
    }

# Save enhanced data
with open('../data/json/post_data_analyzed.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Enhanced analysis saved to: data/json/post_data_analyzed.json")

## Next Steps

1. **Scrape Multiple Posts**: Modify the scraper to process multiple URLs
2. **Time Series Analysis**: Track engagement over time
3. **Comparative Analysis**: Compare posts across different companies
4. **ML Predictions**: Train models to predict engagement
5. **Dashboard**: Create an interactive dashboard with Plotly/Streamlit